# 自定义预训练数据集

## 数据准备

In [ ]:
[
    {
        "text": "xxx"
    },
    {
        "text": "xxx"
    },
    ...
]

注意：
- 每条 text 数据不要太长（分词个数应小于 max_length），以避免在数据处理阶段被截断。
- 为保证数据上下文的一致性，请确保长文本数据在被切分为多个 text 后，json 列表的顺序与实际上下文顺序一致。


## 训练

### 1. 导出config
xtuner/configs/custom_dataset/pretrain/ 目录下有所有支持预训练的配置文件  
xtuner copy-cfg internlm2_7b_full_custom_pretrain_e1 .  
将选用的配置文件复制到当前目录下

### 2. 修改配置文件
```
- data_files = ['/path/to/json/file.json']
+ data_files = ['/path/to/custom_dataset1.json', '/path/to/custom_dataset2.json', ...]
```
目录下的所有数据
```
#######################################################################
#                          PART 1  Settings                           #
#######################################################################
# Data
- data_files = ['/path/to/json/file.json']
+ data_dir = '/dir/to/custom_dataset'

#######################################################################
#                      PART 3  Dataset & Dataloader                   #
#######################################################################
train_dataset = dict(
-   dataset=dict(type=load_dataset, path='json', data_files=data_files),
+   dataset=dict(type=load_dataset, path='json', data_dir=data_dir),
    ...)
```

lora
```
#######################################################################
#                      PART 2  Model & Tokenizer                      #
#######################################################################
model = dict(
    type=SupervisedFinetune,
    use_varlen_attn=use_varlen_attn,
    llm=dict(
        type=AutoModelForCausalLM.from_pretrained,
        pretrained_model_name_or_path=pretrained_model_name_or_path,
        trust_remote_code=True),
+   lora=dict(
+       type=LoraConfig,
+       r=64,
+       lora_alpha=16,
+       lora_dropout=0.1,
+       bias='none',
+       task_type='CAUSAL_LM'))
```

qlora
```
#######################################################################
#                      PART 2  Model & Tokenizer                      #
#######################################################################
model = dict(
    type=SupervisedFinetune,
    use_varlen_attn=use_varlen_attn,
    llm=dict(
        type=AutoModelForCausalLM.from_pretrained,
        pretrained_model_name_or_path=pretrained_model_name_or_path,
        trust_remote_code=True,
+       quantization_config=dict(
+           type=BitsAndBytesConfig,
+           load_in_4bit=True,
+           load_in_8bit=False,
+           llm_int8_threshold=6.0,
+           llm_int8_has_fp16_weight=False,
+           bnb_4bit_compute_dtype=torch.float16,
+           bnb_4bit_use_double_quant=True,
+           bnb_4bit_quant_type='nf4')
    ),
+   lora=dict(
+       type=LoraConfig,
+       r=64,
+       lora_alpha=16,
+       lora_dropout=0.1,
+       bias='none',
+       task_type='CAUSAL_LM')
)
```

### 3. 开始训练
NPROC_PER_NODE=8 xtuner train internlm2_7b_full_custom_pretrain_e1_copy.py --deepspeed deepspeed_zero1  
训得模型将默认保存在 ./work_dirs/，用户可以通过命令 xtuner train --work-dir ${SAVE_PATH} 指定保存路径。

### 4. 模型转换
```
xtuner convert pth_to_hf ${FINETUNE_CFG} ${PTH_PATH} ${SAVE_PATH}
# 例如：xtuner convert pth_to_hf internlm2_7b_full_custom_pretrain_e1_copy.py ./iter_2000.pth ./iter_2000_hf
```

## 对话
全量参数微调
```
xtuner chat ${PATH_TO_LLM} [optional arguments]
# 例如：xtuner chat ./iter_2000_hf --max-new-tokens 512
```
lora或qlora微调
```
xtuner chat ${NAME_OR_PATH_TO_LLM} --adapter {NAME_OR_PATH_TO_ADAPTER} [optional arguments]
# 例如：xtuner chat internlm/internlm2-7b --adapter ./iter_2000_hf --max-new-tokens 512
```

## 模型合并
```
(LLM) xtuner convert merge ${LLM} ${LLM_ADAPTER} ${SAVE_PATH}
```

## 评测
OpenCompass  
https://github.com/InternLM/opencompass